In [2]:
import copy
import time

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from scipy import stats


In [3]:
class Model:

    def __init__(self, model=None, model_path=''):
        """Init method

        :param model: trained PyTorch Model.
        :param model_path: path to trained model.
        """

        self.model_path = model_path
        self.model = torch.load(self.model_path, map_location='cpu')

    def load_model(self):
        if self.model_path != '':
            self.model = torch.load(self.model_path, map_location='cpu')

    def get_output(self, input_tensor):
        return self.model(input_tensor).float()

    def set_eval_mode(self):
        self.model.eval()

    def get_gradient(self, input):
        # Future Support
        raise NotImplementedError("Future Support")


In [4]:

class ExplainerBase(object):

    def __init__(self, data_interface, model_interface):

        self.data_interface = data_interface
        self.model_interface = model_interface

    def generate_counterfactuals(self):

        raise NotImplementedError

    def do_post_sparsity(self):

        raise NotImplementedError

    def do_post_filtering(self):

        raise NotImplementedError



In [5]:

class PlainCF(ExplainerBase):

    def __init__(self, data_interface, model_interface):

        super().__init__(data_interface, model_interface)

    def generate_counterfactuals(self, query_instance, features_to_vary, target = 0.7, feature_weights = None, _lambda = 10,
            optimizer = "adam", lr = 0.01, max_iter = 100):
       
        start_time = time.time()
        query_instance = self.data_interface.prepare_query(query_instance, normalized = True)
        query_instance = torch.FloatTensor(query_instance)

        mask = self.data_interface.get_mask_of_features_to_vary(features_to_vary)
        mask = torch.LongTensor(mask)

        self._lambda = _lambda

        if feature_weights == None:
            feature_weights = torch.ones(query_instance.shape[1])
        else:
            feature_weights = torch.ones(query_instance.shape[0])
            feature_weights = torch.FloatTensor(feature_weights)

        if isinstance(self.data_interface.scaler, MinMaxScaler):
            cf_initialize = torch.rand(query_instance.shape)
        elif isinstance(self.data_interface.scaler, StandardScaler):
            cf_initialize = torch.randn(query_instance.shape)
        else:
            cf_initialize = torch.rand(query_instance.shape)

        cf_initialize = torch.FloatTensor(cf_initialize)
        cf_initialize = mask * cf_initialize + (1 - mask) * query_instance
        
        if optimizer == "adam":
            optim = torch.optim.Adam([cf_initialize], lr)
        else:
            optim = torch.optim.RMSprop([cf_initialize], lr)

        for i in range(max_iter):
            cf_initialize.requires_grad = True
            optim.zero_grad()
            loss = self.compute_loss(cf_initialize, query_instance, target)
            loss.backward()
            cf_initialize.grad = cf_initialize.grad * mask
            optim.step()
            
            if isinstance(self.data_interface.scaler, MinMaxScaler):
                cf_initialize = torch.where(cf_initialize > 1, torch.ones_like(cf_initialize), cf_initialize)
                cf_initialize = torch.where(cf_initialize < 0, torch.zeros_like(cf_initialize), cf_initialize)

            cf_initialize.detach_()

        end_time = time.time()
        running_time = time.time()

        return cf_initialize

    def compute_loss(self, cf_initialize, query_instance, target):
        loss1 = F.relu(target - self.model_interface.get_output(cf_initialize))
        loss2 = torch.sum((cf_initialize - query_instance)**2)
        return self._lambda * loss1 + loss2



In [6]:

def load_adult_income_dataset():
    """Loads adult income dataset from https://archive.ics.uci.edu/ml/datasets/Adult and prepares the data for data analysis based on https://rpubs.com/H_Zhu/235617

    :return adult_data: returns preprocessed adult income dataset.
    """
    # raw_data = np.genfromtxt('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', delimiter=', ', dtype=str)
    raw_data = np.genfromtxt('../artifacts/couterfactual-explanations/adult.data', delimiter=', ', dtype=str)

    #  column names from "https://archive.ics.uci.edu/ml/datasets/Adult"
    column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

    adult_data = pd.DataFrame(raw_data, columns=column_names)


    # For more details on how the below transformations are made, please refer to https://rpubs.com/H_Zhu/235617
    adult_data = adult_data.astype({"age": np.int64, "educational-num": np.int64, "hours-per-week": np.int64})

    adult_data = adult_data.replace({'workclass': {'Without-pay': 'Other/Unknown', 'Never-worked': 'Other/Unknown'}})
    adult_data = adult_data.replace({'workclass': {'Federal-gov': 'Government', 'State-gov': 'Government', 'Local-gov':'Government'}})
    adult_data = adult_data.replace({'workclass': {'Self-emp-not-inc': 'Self-Employed', 'Self-emp-inc': 'Self-Employed'}})
    adult_data = adult_data.replace({'workclass': {'Never-worked': 'Self-Employed', 'Without-pay': 'Self-Employed'}})
    adult_data = adult_data.replace({'workclass': {'?': 'Other/Unknown'}})

    adult_data = adult_data.replace({'occupation': {'Adm-clerical': 'White-Collar', 'Craft-repair': 'Blue-Collar',
                                           'Exec-managerial':'White-Collar','Farming-fishing':'Blue-Collar',
                                            'Handlers-cleaners':'Blue-Collar',
                                            'Machine-op-inspct':'Blue-Collar','Other-service':'Service',
                                            'Priv-house-serv':'Service',
                                           'Prof-specialty':'Professional','Protective-serv':'Service',
                                            'Tech-support':'Service',
                                           'Transport-moving':'Blue-Collar','Unknown':'Other/Unknown',
                                            'Armed-Forces':'Other/Unknown','?':'Other/Unknown'}})

    adult_data = adult_data.replace({'marital-status': {'Married-civ-spouse': 'Married', 'Married-AF-spouse': 'Married', 'Married-spouse-absent':'Married','Never-married':'Single'}})

    adult_data = adult_data.replace({'race': {'Black': 'Other', 'Asian-Pac-Islander': 'Other',
                                           'Amer-Indian-Eskimo':'Other'}})

    adult_data = adult_data[['age','workclass','education','marital-status','occupation','race','gender',
                     'hours-per-week','income']]

    adult_data = adult_data.replace({'income': {'<=50K': 0, '>50K': 1}})

    adult_data = adult_data.replace({'education': {'Assoc-voc': 'Assoc', 'Assoc-acdm': 'Assoc',
                                           '11th':'School', '10th':'School', '7th-8th':'School', '9th':'School',
                                          '12th':'School', '5th-6th':'School', '1st-4th':'School', 'Preschool':'School'}})

    # adult_data = adult_data.rename(columns={'marital-status': 'marital_status', 'hours-per-week': 'hours_per_week'})

    return adult_data


def get_adult_data_info():
    feature_description = {'age':'age',
                        'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
                        'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
                        'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
                        'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
                        'race': 'white or other race?',
                        'gender': 'male or female?',
                        'hours_per_week': 'total work hours per week',
                        'income': '0 (<=50K) vs 1 (>50K)'}
    return feature_description


In [7]:

class Dataset(object):

    """ A data interface for public dataset. """
    def __init__(self, **params):
        """Init Method

        :param dataframe: Pandas Dataframe.
        :param continuous_features: List of names of continous features. The remained features are categorical features.
        :param outcome_name: Outcome feature name.
        :param scaler: The scale type(MinMaxScaler, StandardScaler etc.).
        :param test_size(optional): the proportion of test set split. defaults to 0.2.
        :param random_state: the random state for train_test_split.
        :param custom_preprocessing(optinal): the preprocessing method provided by users.
        :param data_name(optinal): Dataset name.

        """

        if isinstance(params['dataframe'], pd.DataFrame):
            self.data_df = params['dataframe']
        else:
            raise ValueError('The dataframe is not provided')

        if type(params['outcome_name']) is str:
            self.outcome_name = params['outcome_name']
        else:
            raise ValueError('The outcome feature is not provided')

        if 'data_name' in params:
            self.data_name = params['data_name']
        else:
            self.data_name = 'unknown'

        if 'custom_preprocessing' in params:
            self.df = params['custom_preprocessing'](self.data_df.copy())

        if params['continuous_features'] == 'all':
            self.continuous_features_names = self.data_df.columns.tolist()
            self.continuous_features_names.remove(self.outcome_name)
        else:
            if type(params['continuous_features']) is list:
                self.continuous_features_names = params['continuous_features']
            else:
                raise ValueError('The continuous_features is not provided')

        self.categorical_feature_names = [name for name in self.data_df.columns.tolist()
                if name not in self.continuous_features_names + [self.outcome_name]]
        self.feature_names = [name for name in self.data_df.columns.tolist() if name != self.outcome_name]

        self.continuous_features_indices = [self.data_df.columns.get_loc(name) for name in self.continuous_features_names]
        self.categorical_feature_indices = [self.data_df.columns.get_loc(name) for name in self.categorical_feature_names]
        self.outcome_index = [self.data_df.columns.get_loc(self.outcome_name)]

        if 'test_size' in params:
            self.test_size = params['test_size']
        else:
            self.test_size = 0.2

        if 'random_state' in params:
            self.random_state = params['random_state']
        else:
            self.random_state = 0
    
        
        for feature in self.categorical_feature_names:
            self.data_df[feature] = self.data_df[feature].apply(str)
            self.data_df[feature] = self.data_df[feature].astype('category')

        for feature in self.continuous_features_names:
            if self.data_df[feature].dtype == np.float64 or self.data_df[feature].dtype == np.float32:
                self.data_df[feature] = self.data_df[feature].astype(np.float32)
            else:
                self.data_df[feature] = self.data_df[feature].astype(np.int32)
    
        if len(self.categorical_feature_names) > 0:
            one_hot_encoded_data = pd.get_dummies(data = self.data_df, columns = self.categorical_feature_names)
            self.onehot_encoded_names  = one_hot_encoded_data.columns.tolist()
            self.onehot_encoded_names.remove(self.outcome_name)
        else:
            one_hot_encoded_data = self.data_df
            self.onehot_encoded_names = self.feature_names
        
        self.encoded_categorical_feature_indices = self.get_encoded_categorial_feaure_indices()
        # The column name is reordered after one-hot encoding.
        one_hot_x = one_hot_encoded_data[self.onehot_encoded_names].values
        one_hot_y = one_hot_encoded_data[self.outcome_name].values

        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(one_hot_x, one_hot_y, test_size = self.test_size, shuffle = False, random_state = self.random_state)
        
         
        # scale the raw data
        if 'scaler' in params:
            self.scaler = params['scaler']
            self.scaler.fit(self.train_x)
            self.train_scaled_x, self.test_scaled_x = self.scaler.transform(self.train_x.astype(np.float32)), self.scaler.transform(self.test_x.astype(np.float32))

        if len(self.continuous_features_names) > 0:
            self.permitted_range_before_scale = [self.train_x[:, list(range(len(self.continuous_features_names)))].min(0),
                    self.train_x[:, list(range(len(self.continuous_features_names)))].max(0)]
            self.permitted_range_after_scale = [self.train_scaled_x[:, list(range(len(self.continuous_features_names)))].min(0),
                    self.train_scaled_x[:, list(range(len(self.continuous_features_names)))].max(0)]

    def normalize_data(self, input_x):
        try:
            return self.scaler.transform(input_x.astype(np.float32)) # the scaler always returns the float64
        except Exception as e:
            raise e

    def denormalize_data(self, input_x):
        try:
            return self.scaler.inverse_transform(input_x)
        except:
            raise ValueError('scaler is not provided in denormalization')

    def get_mask_of_features_to_vary(self, features_to_vary = ['all']):

        mask = np.ones(len(self.onehot_encoded_names))
        if features_to_vary == ['all']:
            return mask
        else:
            for i in range(len(self.onehot_encoded_names)):
                mask[i] = 0
                for feature in features_to_vary:
                    if self.onehot_encoded_names[i] in self.continuous_features_names:
                        if self.onehot_encoded_names[i] == feature:
                            mask[i] = 1
                            break
                    else:
                        if self.onehot_encoded_names[i].startswith(feature):
                            mask[i] = 1
                            break

            return mask

    def onehot_decode(self, data, prefix_sep = '_'):
        """
        get the original dataframe from dummy onehot encoded data
        """

        if isinstance(data, np.ndarray):
            index = list(range(len(data)))
            data = pd.DataFrame(data = data, index = index, columns = self.onehot_encoded_names)
        
        out = data.copy()
        for feat in self.categorical_feature_names:
            cat_col_values = []
            for val in list(self.data_df[feat].unique()):
                cat_col_values.append(feat + prefix_sep + str(val))

            match_cols = [c for c in data.columns if c in cat_col_values]
            cols, cat_values = [[c.replace(x, "") for c in match_cols] for x in ["", feat + prefix_sep]]
            out[feat] = pd.Categorical(np.array(cat_values)[np.argmax(data[cols].values, axis = 1)])
            out.drop(cols, axis = 1, inplace = True)
        
        # The columns are shuffled after one-hot encoding and decoding. Here we re-order the columns as the input dataframe.
        
        pairs = list(zip(self.continuous_features_indices + self.categorical_feature_indices, self.continuous_features_names + self.categorical_feature_names))
        sorted_pairs = sorted(pairs, key = lambda t:t[0])
        sorted_indices, sorted_columns = list(zip(*sorted_pairs))

        decoded_data = out[list(sorted_columns)]
        for i in decoded_data.columns:
            if self.data_df[i].dtypes == np.int32 or self.data_df[i].dtypes == np.int64:
                decoded_data[i] = decoded_data[i].round().astype(self.data_df[i].dtypes, copy = False)

        return decoded_data

    def get_encoded_categorial_feaure_indices(self):

        cols = []
        for col_parent in self.categorical_feature_names:
            temp = [self.onehot_encoded_names.index(col) for col in self.onehot_encoded_names if col.startswith(col_parent) and col not in self.continuous_features_names]
            cols.append(temp)

        return cols
    
    def get_quantiles_from_data(self, quantile = 0.05, normalized = True):

        quantile = np.zeros(len(self.onehot_encoded_names))
        if normalized:
            quantile = []
        else:
            quantile = []

        return quantile

    def compute_continuous_percentile_shift(self, source, target, features_to_vary, normalized = False, method = 'sum'):

        continuous_shift = np.zeros(len(self.continuous_features_names)).astype(np.float32)
        for i in range(len(self.continuous_features_names)):
            if self.continuous_features_names[i] in features_to_vary:
                if normalized:
                    source_percentile = stats.percentileofscore(self.train_scaled_x[:, i], source[:, i])
                    target_percentile = stats.percentileofscore(self.train_scaled_x[:, i], target[:, i])
                    continuous_shift[i] = np.abs(source_percentile - target_percentile)
                else:
                    source_percentile = stats.percentileofscore(self.train_x[:, i], source[:, i])
                    target_percentile = stats.percentileofscore(self.train_x[:, i], target[:, i])
                    continuous_shift[i] = np.abs(source_percentile - target_percentile)
            else:
                continue

        continuous_shift = continuous_shift / 100
        if method == "sum":
            score = np.mean(continuous_shift)
        elif method == "max":
            score = np.max(continuous_shift)

        return score

    def compute_categorical_changes(self, source, target):

        source, target = self.onehot_decode(source), self.onehot_decode(target)
        match = (source[self.categorical_feature_names].values != target[self.categorical_feature_names].values)
        categorical_change = np.mean(match)
        return categorical_change

    def compute_sparsity(self, source, target):
        return (source == target).values.sum()
        
    def compute_actionability_score(self, source, target, features_to_vary, continous_rules, categorical_rules):
        scores = np.zeros(len(features_to_vary))
        for i in range(len(features_to_vary)):
            feature = features_to_vary[i]
            if feature in continous_rules:
                source_value = source.iloc[0][feature]
                target_value = target[feature]
                
                if continous_rules[feature]:
                    if target_value > source_value:
                        scores[i] = 1
                    elif target_value == source_value:
                        scores[i] = 0
                    else:
                        scores[i] = -1
                else:
                    if target_value < source_value:
                        scores[i] = 1
                    elif target_value == source_value:
                        scores[i] = 0
                    else:
                        scores[i] = -1

            elif feature in categorical_rules:
                source_value = categorical_rules[feature][source.iloc[0][feature]]
                target_value = categorical_rules[feature][target[feature]]
                if target_value > source_value:
                    scores[i] = 1
                elif target_value == source_value:
                    scores[i] = 0
                else:
                    scores[i] = -1
            else:
                continue

        score = scores.sum() / (target != source).values.sum()

        return score

    def prepare_query(self, query_instance, normalized = False):
        
        if isinstance(query_instance, list):
            test = pd.DataFrame(query_instance, orient = 'index', columns = self.feature_names)
        elif isinstance(query_instance, dict):
            test = pd.DataFrame.from_records([query_instance])
        else:
            raise ValueError("unsupported data type of query_instance")
    
        tmp  = np.zeros((1, len(self.onehot_encoded_names)))
        onehot_test = pd.DataFrame(tmp, columns = self.onehot_encoded_names)

        for name, content in test.items():
            if content.dtype == np.float64 or content.dtype == np.float32:
                onehot_test[name] = test[name]
            elif content.dtype == np.int64 or content.dtype == np.int32:
                onehot_test[name] = test[name]
            else:
                onehot_name = name + "_" + content[0]
                onehot_test[onehot_name] = 1

        onehot_test = onehot_test.values
        if normalized:
            onehot_test = self.normalize_data(onehot_test)

        return onehot_test



In [8]:
data_df = load_adult_income_dataset()
data_df

,age,workclass,education,marital-status,occupation,race,gender,hours-per-week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0
...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc,Married,Service,White,Female,38,0
32557,40,Private,HS-grad,Married,Blue-Collar,White,Male,40,1
32558,58,Private,HS-grad,Widowed,White-Collar,White,Female,40,0
32559,22,Private,HS-grad,Single,White-Collar,White,Male,20,0


In [9]:
income_df = data_df[:500]
income_df

,age,workclass,education,marital-status,occupation,race,gender,hours-per-week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0
...,...,...,...,...,...,...,...,...,...
495,41,Private,HS-grad,Divorced,White-Collar,White,Female,36,0
496,20,Private,Some-college,Single,Blue-Collar,White,Male,40,0
497,23,Private,School,Single,Service,White,Male,40,0
498,26,Private,Some-college,Single,Professional,White,Female,35,0


In [10]:

# ['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
d = Dataset(dataframe = income_df, continuous_features = ['age', 'hours-per-week'], outcome_name = 'income', scaler = MinMaxScaler())


/tmp/ipykernel_10438/618290084.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data_df[feature] = self.data_df[feature].apply(str)
/tmp/ipykernel_10438/618290084.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data_df[feature] = self.data_df[feature].astype('category')
/tmp/ipykernel_10438/618290084.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [12]:
clf = Model(model_path = '../artifacts/couterfactual-explanations/adult.pth')
cf = PlainCF(d, clf)

In [13]:
get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [14]:
test_instance = {'age': 57, 'workclass' : 'Self-Employed', 'education' : 'Some-college', 'marital-status':'Married', 'occupation':'Service', 'race':'White', 'gender':'Male', 'hours-per-week':60}

In [15]:
results = cf.generate_counterfactuals(test_instance, features_to_vary = ['age', 'education', 'hours-per-week'])

In [16]:
results = results.detach().numpy()
results = d.denormalize_data(results)
results = d.onehot_decode(results)
results


,age,workclass,education,marital-status,occupation,race,gender,hours-per-week
0,20,Self-Employed,Some-college,Married,Service,White,Male,68


In [17]:
test_query = d.prepare_query(test_instance, normalized = True)
test_query = torch.FloatTensor(test_query)
clf.get_output(test_query)

tensor([[0.7876]], grad_fn=<SigmoidBackward0>)

In [18]:
result_query = d.prepare_query(results.to_dict('records')[0], normalized = True)
result_query = torch.FloatTensor(result_query)
clf.get_output(result_query) 

tensor([[0.2100]], grad_fn=<SigmoidBackward0>)